In [1]:
import os
import json
import csv

def process_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    tool_name = os.path.splitext(os.path.basename(file_path))[0]
    
    durations = data['durations']
    total_duration = durations['total']['duration']
    setup_duration = durations['toolSetup']['duration']
    avg_measurement_duration = durations['measurements']['average']['duration']
    
    measurements = data['measurements']
    avg_speeds = {
        '1MB': sum(m['fileTransfers']['1MB_buffer']['speedDownload'] for m in measurements) / len(measurements),
        '2MB': sum(m['fileTransfers']['2MB_buffer']['speedDownload'] for m in measurements) / len(measurements),
        '3MB': sum(m['fileTransfers']['3MB_buffer']['speedDownload'] for m in measurements) / len(measurements)
    }
    
    return {
        'Tool': tool_name,
        'Total Duration': total_duration,
        'Setup Duration': setup_duration,
        'Avg Measurement Duration': avg_measurement_duration,
        '1MB Avg Speed': avg_speeds['1MB'],
        '2MB Avg Speed': avg_speeds['2MB'],
        '3MB Avg Speed': avg_speeds['3MB']
    }

def process_directory(directory):
    results = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            results.append(process_json_file(file_path))
    return results

def create_csv(results, output_file):
    fieldnames = ['Tool', 'Total Duration', 'Setup Duration', 'Avg Measurement Duration', 
                  '1MB Avg Speed', '2MB Avg Speed', '3MB Avg Speed']
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)

def main():
    directory = input("Enter the directory path containing JSON files: ")
    results = process_directory(directory)
    
    csv_output = os.path.join(directory, 'network_measurements.csv')
    create_csv(results, csv_output)
    print(f"CSV file created: {csv_output}")

if __name__ == "__main__":
    main()

KeyError: '1MB_buffer'